In [1]:
# load libraries
import os
import glob
import pandas as pd
import numpy as np

The list "sessions" contains six elements, each of which represents students' behaviors and activities in a particular session

In [2]:
sessions = []
for root, dirs, files in os.walk("../EPM_dataset/Data/Processes/", topdown=False):
    if files:
        session = []
        for file in files:
            path = os.path.join(root, file)
            log = pd.read_csv(path, sep=",", header=None)
            session.append(log)

        session_pd = pd.concat(session)
        session_pd.columns = ["session", "student_id", "exercise", 
                      "activity",'start_time','end_time',
                     'idle_time','mouse_wheel','mouse_wheel_click','mouse_click_left',
                     'mouse_click_right','mouse_movement','keystroke']
        sessions.append(session_pd)

sessions[0].head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 1072: invalid start byte

In [74]:
# The sessions array contains all data session by session, but not orderly.
for i, session in enumerate(sessions):
    session_num = session['session'].unique()[0]
    print(f"{i}th element in the sessions list represents Session{session_num}")

0th element in the sessions list represents Session4
1th element in the sessions list represents Session3
2th element in the sessions list represents Session2
3th element in the sessions list represents Session5
4th element in the sessions list represents Session6
5th element in the sessions list represents Session1


In [75]:
# Insert the ordered session data to the data_list array.
data_list = [0]*7
session_num = 0
while sessions:
    session_num = int(sessions[0]['session'].unique())
    data_list[session_num] = sessions.pop(0)
    session_num +=1

# confirm
for i, session in enumerate(data_list):
    if i == 0:
        continue
    session_num = session['session'].unique()[0]
    print(f"{i}th element in the sessions list represents Session{session_num}")    

1th element in the sessions list represents Session1
2th element in the sessions list represents Session2
3th element in the sessions list represents Session3
4th element in the sessions list represents Session4
5th element in the sessions list represents Session5
6th element in the sessions list represents Session6


In [76]:
# print session1 data
data_list[1].head(10)

,session,student_id,exercise,activity,start_time,end_time,idle_time,mouse_wheel,mouse_wheel_click,mouse_click_left,mouse_click_right,mouse_movement,keystroke
0,1,59,Es,Other,2.10.2014 11:24:0,2.10.2014 11:24:3,421,0,0,2,0,106,0
1,1,59,Es,Deeds,2.10.2014 11:24:5,2.10.2014 11:25:42,4573639,0,0,2,0,107,0
2,1,59,Es,Blank,2.10.2014 11:25:43,2.10.2014 11:25:44,0,0,0,2,0,123,0
3,1,59,Es_1_1,Study_Es_1_1,2.10.2014 11:25:45,2.10.2014 11:26:46,1652751,0,0,2,0,107,0
4,1,59,Es_1_1,Deeds,2.10.2014 11:26:47,2.10.2014 11:26:47,0,0,0,2,0,18,0
5,1,59,Es_1_1,Study_Es_1_1,2.10.2014 11:26:48,2.10.2014 11:26:57,18529,0,0,4,2,390,0
6,1,59,Es_1_1,Blank,2.10.2014 11:26:58,2.10.2014 11:26:58,0,0,0,2,0,72,0
7,1,59,Es_1_1,Study_Es_1_1,2.10.2014 11:26:59,2.10.2014 11:27:0,0,0,0,2,0,108,0
8,1,59,Es_1_1,Blank,2.10.2014 11:27:1,2.10.2014 11:27:2,422,0,0,6,0,85,0
9,1,59,Es_1_1,Deeds,2.10.2014 11:27:3,2.10.2014 11:27:4,16,0,0,1,0,99,0


In [77]:
# drop irrelevant columns and give simpler column names
for i, session in enumerate(data_list):
    if i == 0: continue
    data_list[i] = session.drop(columns = ['session', 'exercise', 'start_time', 'end_time'])
    data_list[i].columns = ["ID", "ACT",
                            'DUR','MW','MWC','MCL',
                            'MCR','MM','KS']
    
# verify    
data_list[1].head(10)

,ID,ACT,DUR,MW,MWC,MCL,MCR,MM,KS
0,59,Other,421,0,0,2,0,106,0
1,59,Deeds,4573639,0,0,2,0,107,0
2,59,Blank,0,0,0,2,0,123,0
3,59,Study_Es_1_1,1652751,0,0,2,0,107,0
4,59,Deeds,0,0,0,2,0,18,0
5,59,Study_Es_1_1,18529,0,0,4,2,390,0
6,59,Blank,0,0,0,2,0,72,0
7,59,Study_Es_1_1,0,0,0,2,0,108,0
8,59,Blank,422,0,0,6,0,85,0
9,59,Deeds,16,0,0,1,0,99,0


In [78]:
# recode values to shrink the size of variables
for i, session in enumerate(data_list):
    if i == 0: continue
    session['ACT'] = session['ACT'].replace(regex=['TextEditor\w+','Deeds\w+', 'Study\w+', '(?i)FSM_\w+'], 
                         value=['TextEditor','Deeds','Study','FSM'])
    session['ACT'] = session['ACT'].str.strip()


In [79]:
# verify
for i, session in enumerate(data_list):
    if i == 0: continue
    print(i, session.ACT.unique(), len(session.ACT.unique()))

1 ['Other' 'Deeds' 'Blank' 'Study' 'Properties' 'Diagram' 'Aulaweb'
 'TextEditor' 'FSM'] 9
2 ['Other' 'Aulaweb' 'TextEditor' 'Study' 'Deeds' 'Blank' 'Diagram' 'FSM'
 'Properties'] 9
3 ['Other' 'Aulaweb' 'Study' 'Blank' 'TextEditor' 'Deeds' 'Properties'
 'Diagram' 'FSM'] 9
4 ['Other' 'Aulaweb' 'Blank' 'TextEditor' 'Deeds' 'Diagram' 'Study'
 'Properties' 'FSM'] 9
5 ['Other' 'TextEditor' 'Study' 'Deeds' 'Blank' 'Properties' 'Diagram'
 'Aulaweb' 'FSM'] 9
6 ['Other' 'Blank' 'Aulaweb' 'Study' 'FSM' 'Deeds' 'Properties' 'TextEditor'
 'Diagram'] 9


In [80]:
# sum figures for each features of each student
for i, session in enumerate(data_list):
    if i == 0: continue
    data_list[i] = session.groupby(['ID','ACT']).sum()
    
# verify
data_list[1].head(10)

DUR   MW  MWC   MCL  MCR      MM   KS
ID ACT                                                   
1  Aulaweb      3647532   27    0    80    0    8873    0
   Blank          33274    0    0   207   30   14322    5
   Deeds       32405168   21    0  1636  197  123316  152
   Diagram     57068818    9    0  1141   44   78390    4
   Other         673720   54    0   252    6   21783   71
   Properties     72754    0    0   104    0    6096   54
   Study        3578043   74    0   194    6   28600    0
   TextEditor   1165152  265    0   702   11   43592  898
2  Aulaweb        90533    0    0     0    0       0    0
   Blank           5119    0    0     0    0       0    5

In [81]:
# transform variables from two dimensions to one
for i, session in enumerate(data_list):
    if i == 0: continue
    data_list[i] = session.pivot_table(index = ['ID'],
                                       columns = 'ACT',
                                       values = ['DUR', 'MW', 'MWC', 'MCR', 'MCL', 'MM', 'KS'])
    data_list[i].columns = ['_'.join(col) for col in data_list[i].columns.values]

# verify
data_list[1].head(10)    

,DUR_Aulaweb,DUR_Blank,DUR_Deeds,DUR_Diagram,DUR_FSM,DUR_Other,DUR_Properties,DUR_Study,DUR_TextEditor,KS_Aulaweb,...,MW_TextEditor,MWC_Aulaweb,MWC_Blank,MWC_Deeds,MWC_Diagram,MWC_FSM,MWC_Other,MWC_Properties,MWC_Study,MWC_TextEditor
ID,,,,,,,,,,,,,,,,,,,,,
1,3647532.0,33274.0,32405168.0,57068818.0,NaN,673720.0,72754.0,3578043.0,1165152.0,0.0,...,265.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,90533.0,5119.0,153036710.0,15821086.0,NaN,1144159.0,22007.0,1374319.0,12295064.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,467029.0,40315.0,17320071.0,7786970.0,NaN,3576169.0,24706.0,45434441.0,3644990.0,6.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
5,1302787.0,5292254.0,28960131.0,9919495.0,NaN,1618069.0,105690.0,572534.0,2609364.0,189.0,...,363.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
7,937.0,5527.0,122404677.0,58494163.0,NaN,1006004.0,15979.0,2572151.0,5644434.0,0.0,...,0.0,0.0,0.0,12.0,2.0,NaN,0.0,0.0,0.0,4.0
9,9001068.0,348144.0,2809682.0,1220659.0,NaN,1081617.0,3666.0,668826.0,1161889.0,0.0,...,298.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
10,NaN,30.0,215751705.0,15104244.0,NaN,6152924.0,19868.0,NaN,22242360.0,NaN,...,549.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
11,6399655.0,182271.0,77054626.0,12587415.0,NaN,8937181.0,3999.0,1230143.0,99756012.0,0.0,...,136.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
12,12753848.0,12312.0,7473932.0,26808385.0,NaN,7350044.0,25786.0,1694901.0,2792364.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0


In [82]:
# filter out insignificant columns
for i, session in enumerate(data_list):
    if i == 0: continue
        
    columns_to_be_deleted = []
    for col_name in session.columns:
        if len(session[col_name].unique()) < 6:
            columns_to_be_deleted.append(col_name)
    
    data_list[i] = session.drop(columns = columns_to_be_deleted)
    print(f"{columns_to_be_deleted} are deleted in session {i}")

['DUR_FSM', 'KS_FSM', 'MCL_FSM', 'MCR_FSM', 'MCR_Properties', 'MM_FSM', 'MW_FSM', 'MW_Properties', 'MWC_Blank', 'MWC_Deeds', 'MWC_Diagram', 'MWC_FSM', 'MWC_Other', 'MWC_Properties', 'MWC_Study', 'MWC_TextEditor'] are deleted in session 1
['DUR_FSM', 'KS_FSM', 'MCL_FSM', 'MCR_FSM', 'MCR_Properties', 'MM_FSM', 'MW_FSM', 'MW_Properties', 'MWC_Blank', 'MWC_Deeds', 'MWC_Diagram', 'MWC_FSM', 'MWC_Other', 'MWC_Properties', 'MWC_TextEditor'] are deleted in session 2
['DUR_FSM', 'KS_FSM', 'MCL_FSM', 'MCR_Aulaweb', 'MCR_FSM', 'MCR_Properties', 'MCR_Study', 'MM_FSM', 'MW_FSM', 'MW_Properties', 'MWC_Blank', 'MWC_Deeds', 'MWC_Diagram', 'MWC_FSM', 'MWC_Other', 'MWC_Properties'] are deleted in session 3
['DUR_FSM', 'KS_FSM', 'MCL_FSM', 'MCR_FSM', 'MW_FSM', 'MWC_Aulaweb', 'MWC_Blank', 'MWC_Deeds', 'MWC_Diagram', 'MWC_FSM', 'MWC_Other', 'MWC_Properties', 'MWC_TextEditor'] are deleted in session 4
['DUR_FSM', 'KS_FSM', 'MCL_FSM', 'MCR_Aulaweb', 'MCR_FSM', 'MCR_Properties', 'MM_FSM', 'MW_FSM', 'MWC_Aulaw

In [83]:
# verify
for i, session in enumerate(data_list):
    if i == 0: continue
    print(f"In session {i}, the number of columns is reduced to {len(session.columns)} from 63")

In session 1, the number of columns is reduced to 47 from 63
In session 2, the number of columns is reduced to 48 from 63
In session 3, the number of columns is reduced to 47 from 63
In session 4, the number of columns is reduced to 50 from 63
In session 5, the number of columns is reduced to 46 from 63
In session 6, the number of columns is reduced to 54 from 63


In [84]:
# verify
data_list[1].head(10) 

,DUR_Aulaweb,DUR_Blank,DUR_Deeds,DUR_Diagram,DUR_Other,DUR_Properties,DUR_Study,DUR_TextEditor,KS_Aulaweb,KS_Blank,...,MM_Study,MM_TextEditor,MW_Aulaweb,MW_Blank,MW_Deeds,MW_Diagram,MW_Other,MW_Study,MW_TextEditor,MWC_Aulaweb
ID,,,,,,,,,,,,,,,,,,,,,
1,3647532.0,33274.0,32405168.0,57068818.0,673720.0,72754.0,3578043.0,1165152.0,0.0,5.0,...,28600.0,43592.0,27.0,0.0,21.0,9.0,54.0,74.0,265.0,0.0
2,90533.0,5119.0,153036710.0,15821086.0,1144159.0,22007.0,1374319.0,12295064.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,467029.0,40315.0,17320071.0,7786970.0,3576169.0,24706.0,45434441.0,3644990.0,6.0,5.0,...,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1302787.0,5292254.0,28960131.0,9919495.0,1618069.0,105690.0,572534.0,2609364.0,189.0,2.0,...,25706.0,29833.0,70.0,12.0,13.0,19.0,19.0,121.0,363.0,0.0
7,937.0,5527.0,122404677.0,58494163.0,1006004.0,15979.0,2572151.0,5644434.0,0.0,3.0,...,4960.0,13078.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9001068.0,348144.0,2809682.0,1220659.0,1081617.0,3666.0,668826.0,1161889.0,0.0,0.0,...,8974.0,19615.0,51.0,0.0,7.0,0.0,141.0,97.0,298.0,0.0
10,NaN,30.0,215751705.0,15104244.0,6152924.0,19868.0,NaN,22242360.0,NaN,0.0,...,NaN,34808.0,NaN,0.0,24.0,0.0,10.0,NaN,549.0,NaN
11,6399655.0,182271.0,77054626.0,12587415.0,8937181.0,3999.0,1230143.0,99756012.0,0.0,5.0,...,16881.0,56276.0,37.0,5.0,36.0,38.0,21.0,65.0,136.0,0.0
12,12753848.0,12312.0,7473932.0,26808385.0,7350044.0,25786.0,1694901.0,2792364.0,0.0,0.0,...,11076.0,9451.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# replace nan to 0
for i, session in enumerate(data_list):
    if i == 0: continue
    data_list[i] = session.fillna(0)
    
# verify
data_list[1].head(10) 

,DUR_Aulaweb,DUR_Blank,DUR_Deeds,DUR_Diagram,DUR_Other,DUR_Properties,DUR_Study,DUR_TextEditor,KS_Aulaweb,KS_Blank,...,MM_Study,MM_TextEditor,MW_Aulaweb,MW_Blank,MW_Deeds,MW_Diagram,MW_Other,MW_Study,MW_TextEditor,MWC_Aulaweb
ID,,,,,,,,,,,,,,,,,,,,,
1,3647532.0,33274.0,32405168.0,57068818.0,673720.0,72754.0,3578043.0,1165152.0,0.0,5.0,...,28600.0,43592.0,27.0,0.0,21.0,9.0,54.0,74.0,265.0,0.0
2,90533.0,5119.0,153036710.0,15821086.0,1144159.0,22007.0,1374319.0,12295064.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,467029.0,40315.0,17320071.0,7786970.0,3576169.0,24706.0,45434441.0,3644990.0,6.0,5.0,...,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1302787.0,5292254.0,28960131.0,9919495.0,1618069.0,105690.0,572534.0,2609364.0,189.0,2.0,...,25706.0,29833.0,70.0,12.0,13.0,19.0,19.0,121.0,363.0,0.0
7,937.0,5527.0,122404677.0,58494163.0,1006004.0,15979.0,2572151.0,5644434.0,0.0,3.0,...,4960.0,13078.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9001068.0,348144.0,2809682.0,1220659.0,1081617.0,3666.0,668826.0,1161889.0,0.0,0.0,...,8974.0,19615.0,51.0,0.0,7.0,0.0,141.0,97.0,298.0,0.0
10,0.0,30.0,215751705.0,15104244.0,6152924.0,19868.0,0.0,22242360.0,0.0,0.0,...,0.0,34808.0,0.0,0.0,24.0,0.0,10.0,0.0,549.0,0.0
11,6399655.0,182271.0,77054626.0,12587415.0,8937181.0,3999.0,1230143.0,99756012.0,0.0,5.0,...,16881.0,56276.0,37.0,5.0,36.0,38.0,21.0,65.0,136.0,0.0
12,12753848.0,12312.0,7473932.0,26808385.0,7350044.0,25786.0,1694901.0,2792364.0,0.0,0.0,...,11076.0,9451.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
# save to csv
for i, session in enumerate(data_list):
    if i == 0: continue
    file_name = "variables_session_" + str(i) + ".csv"
    session.to_csv('../EPM_dataset/Data/Seyoung/' + file_name)